In [8]:
import random
import colorsys

import numpy as np
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
from PIL import ImageEnhance
import os
import datetime

In [107]:
CAPTCHA_CHARS = list('0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ')
# Difficult version
CAPTCHA_CHARS_HARD = ['o0OQD', '8BE', 'g9q7', '1IiljJ', '2zZ', '5Ss', 'A4', 'MNmnhH']

def random_color():
    return tuple(map(lambda x: int(x*255), colorsys.hsv_to_rgb(random.random()-0.3, random.randint(8, 10)/10, random.randint(0, 5)/5)))

def random_captcha_str(length):
    return "".join(random.choices(CAPTCHA_CHARS, k=length))

def random_captcha_str_hard(length):
    chars_like = random.choice(CAPTCHA_CHARS_HARD)
    weights_like = np.ones(len(chars_like), dtype=float)
    weights_other = np.zeros(len(CAPTCHA_CHARS), dtype=float) + (1 / len(CAPTCHA_CHARS) * 1.7)
    weights = np.concatenate((weights_like, weights_other), axis=0)
    concat = np.concatenate((list(chars_like), CAPTCHA_CHARS), axis=0)
    return "".join(random.choices(concat, weights=weights, k=length))

random_captcha_str(4), random_captcha_str_hard(4)

('NrGw', 'gd5U')

In [108]:
def generate_picture(width=80, height=30):
    image = Image.new('RGB', (width, height), color=(226, 239, 248))
    return image

In [109]:
def draw_captcha(image, captcha, font_size=25, line_count=5):
    draw = ImageDraw.Draw(image)
    # 获取一个font字体对象参数是ttf的字体文件的目录，以及字体的大小
    font_file = os.path.join('./fonts/font3.ttf')
    font = ImageFont.truetype(font_file, size=font_size, index=0)
    for i in range(line_count):
        x1 = random.randint(0, image.width)
        x2 = random.randint(0, image.width)
        y1 = random.randint(0, image.height)
        y2 = random.randint(0, image.height)
        # 随机划线
        draw.line((x1, y1, x2, y2), fill=random_color())
    for i, random_char in enumerate(list(captcha)):
        draw.text((2+i*20, -3), random_char, random_color(), font=font)

    #对比度增强
    enh_con = ImageEnhance.Contrast(image)
    contrast = 1.1
    image = enh_con.enhance(contrast)

    #锐度增强
    enh_sha = ImageEnhance.Sharpness(image)
    sharpness = 3.5
    image = enh_sha.enhance(sharpness)

    return image

draw_captcha(generate_picture(), random_captcha_str_hard(4))

In [113]:
if __name__ == '__main__':
    now_stamp = int(datetime.datetime.now().timestamp())
    for _ in range(1000):
        now_stamp += 1
        label = random_captcha_str_hard(4)
        draw_captcha(generate_picture(), label).save(f'../sample/valid_hard/{label}_{now_stamp}.jpg')